In [15]:
import sklearn
import numpy as np
from keras.datasets import cifar10, mnist, imdb
from random import randrange
import time
from sklearn.model_selection import train_test_split
from keras import layers, models, optimizers, backend, metrics, callbacks
import codecs

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 12) # set default size of plots

In [2]:
import csv

In [4]:
texts = []
labels = []
with open('labeledTrainData.tsv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    _ = next(reader)
    for l in reader:
        texts.append(l[2])
        labels.append(l[1])

In [8]:
from keras.preprocessing import text, sequence
from nltk.tokenize import word_tokenize

In [12]:
class Vocab(object):
    w2i = None
    i2w = None

In [13]:
def load_glove(filename, embedding_size=300):
    vocabulary_mapping = {'PAD': 0}
    temp = np.zeros(embedding_size)
    embeddings_list = [[0] * embedding_size]
    id = 1
    with codecs.open(filename, 'rb', 'utf-8') as glove_file:
        for line in glove_file:
            items = line.strip().split()

            if len(items) != embedding_size + 1:
                continue

            token = items[0]
            embedding = np.array([float(i) for i in items[1:]])
            temp += embedding
            embeddings_list.append(embedding)
            vocabulary_mapping[token] = id
            id += 1

    vocabulary_mapping['<UNK>'] = id
    temp /= len(embeddings_list) - 2
    embeddings_list.append(temp)
    embeddings = np.array(embeddings_list)
    vocab = Vocab()
    vocab.i2w = {v: k for k, v in vocabulary_mapping.items()}
    vocab.w2i = vocabulary_mapping
    print('glove was loaded')
    return embeddings, vocab

In [17]:
embeddings, vocab = load_glove('/mnt/Science/glove/glove.6B.100d.txt', 100)

glove was loaded


In [18]:
def sent_to_id_vec(sent, vocab, max_len=42, mode='tokenize'):
    sent = sent.lower()

    if mode == 'tokenize':
        tokens = word_tokenize(sent)
    elif mode == 'split':
        tokens = sent.split()
    else:
        raise Error()

    if len(tokens) > max_len:
        tokens = (t for i, t in enumerate(tokens) if i < max_len)

    result = []

    for token in tokens:
        if token in vocab.w2i:
            result.append(vocab.w2i[token])
        else:
            result.append(vocab.w2i['<UNK>'])

    return result

In [ ]:
sequences = []
for t in texts:
    temp = sent_to_id_vec(t, vocab)
    sequences.append(temp)